In [12]:
import requests
from bs4 import BeautifulSoup
import shutil
from IPython.display import Image
from Crypto.Cipher import AES
import base64
import thread
import cn_class_4alogin

_username='sh_fuy'
_pwd='m2mnew123'
import time

#AES加密
def cn_encrypted(text,key):
    BS = AES.block_size
    pad =lambda s: s +(BS - len(s)% BS)* chr(BS - len(s)% BS)
    cipher = AES.new(key,AES.MODE_CBC, key)
    encrypted = base64.b64encode(cipher.encrypt(pad(text)))
    return encrypted 

    
r=requests.session()
header={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:44.0) Gecko/20100101 Firefox/44.0','Accept-Language':'en-US,en;q=0.5'}

#获取主页
s=r.get('http://120.198.253.23:8080/iotpro/',headers=header)
soup=BeautifulSoup(s.text,'lxml')
csrftoken=type(soup.select('#csrftoken')[0].attrs['value'])


#获取genPwToken
s=r.get('http://120.198.253.23:8080/iotpro/genPwToken',headers=header)
key=BeautifulSoup(s.text,'lxml').text.replace('"','')
username=cn_encrypted(_username,key)
pwd=cn_encrypted(_pwd,key)


#获取验证码
# cn_checkcode(r,'1')
s=r.get('http://120.198.253.23:8080/iotpro/captcha-image',headers=header,stream=True)
f = open('FuYou_iot_check.png','wb')
shutil.copyfileobj(s.raw,f)
f.close() 
Image('FuYou_iot_check.png')



In [13]:
checkcode=input('验证码：')
#login
parma={'csrftoken':csrftoken,'username':username,'password':pwd,'_username':'','_password':'','checkcode':checkcode}
r.post('http://120.198.253.23:8080/iotpro/login',data=parma,headers=header)



验证码：8619


<Response [200]>

In [14]:
#流量告警查明细
cn_date= time.strftime('%Y-%m-%d')
parma={'isload':"true",'type':"alarm_stream",'userType':"",'msisdnOrSim':"",'alarmType':"",'startTime':cn_date[:-3]+'01','endTime':cn_date}
s=r.post('http://120.198.253.23:8080/iotpro/alarm/stream',data=parma,headers=header)
soup=BeautifulSoup(s.text,'lxml')
for i in soup.select('.to_url'):
    print i.text

15728201507
15728201507


In [ ]:
#登录BOSS
import cn_class_4alogin

json2=cn_class_4alogin.cn_4ALogin(r,1,'192.168.0.4','8081')
cn_class_4alogin.cn_BossLogin(r,json2,1,'192.168.0.4','8081')


已经发送短信验证码


In [135]:

def cn_cookies(cookies):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='null'
        cookies['com.huawei.boss.CURRENT_MENUID']='123456'
        cookies['com.huawei.boss.CURRENT_TAB']='123456'
        cookies['com.huawei.boss.CURRENT_USER']='com.huawei.boss.NO_CURRENT_USER'
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        return cookies

def cn_cookies1(cookies,cn_phoneNum):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='undefined'
        cookies['com.huawei.boss.CURRENT_MENUID']='100110121062'
        cookies['com.huawei.boss.CURRENT_TAB']='BOSS%5E'+cn_phoneNum+'%5E100110121062%7E'+cn_phoneNum
        cookies['com.huawei.boss.CURRENT_USER']=cn_phoneNum
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        cookies['MACAddr']='08:00:27:A2:E7:39'
        cookies['sDNSName']='gzshizhongxia'
        return cookies

    
def cn_main_data(r,cn_phoneNum,cn_pwd,isProxy,proxyIp,proxyPort):
    proxy={}
    if (isProxy==1):
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
        
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
    header2={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate','X-Requested-With':'XMLHttpRequest'}
    header3={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'image/jpeg,application/x-ms-application,image/gif,application/xaml+xml,image/pjpeg, application/x-ms-xbap','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}

    # # ---------号码认证---------
    cookies=cn_cookies(r.cookies)
    parmas={'method':'qryCustInfo','servNumber':cn_phoneNum,'authCheckMode':'AuthCheckZ','verifyCode':'','pswd':cn_pwd,'certType':'IdCard','certID':'','rndPswd':'','custType':'PersonCustomer','domainType':'','isCert2G':'','ONLYLOGIN':'onlyLogin','withoutPassValidate':'true'}
    result=r.post('http://10.252.17.241/ngcustcare/custlogin/qryCustInfo.action',cookies=cookies,data=parmas,headers=header2,proxies=proxy)
    s=result.text
    cn_curCustId=s[s.find('curCustId')+12:s.find('curCustId')+25]
    # # ---------获取最重要数据---------
    cookies=cn_cookies1(r.cookies,cn_phoneNum)
    s=r.get('http://10.252.17.241/ngcustcare/uniteview/uviewtwo/uvDisper.action?currentTabID=BOSS^'+cn_phoneNum+'^100110121062~'+cn_phoneNum,cookies=cookies,headers=header3,proxies=proxy)
    return s.text
    # # --------------------------

        
header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
header2={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate','X-Requested-With':'XMLHttpRequest'}
header3={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'image/jpeg,application/x-ms-application,image/gif,application/xaml+xml,image/pjpeg, application/x-ms-xbap','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}

#查号码状态和流量使用情况 US10：正使用  US30：停用
soup=BeautifulSoup(cn_main_data(r,'15728201507','',1,'192.168.0.5','8081'),'lxml')
csm_Status=soup.find(attrs={'name':'csm_Status'}).attrs['value']
#获取流量
print csm_Status
proxy={'http':'192.168.0.5:8081','https':'192.168.0.5:8081'}
url='http://10.252.17.241/nguniteview/layoutAction.do?method=showView&ownerType=1&viewId=208'
param={'width':'783','height':'14'}
cookies=cn_cookies1(r.cookies,'15728201507')
s=r.post(url,proxies=proxy,headers=header2,data=param,cookies=cookies)
print s.text


US10
{"brief":"请求数据异常，无法获取视图的具体信息","detail":"An Error has occurred in this application.","flag":-1,"success":false,"title":"视图请求失败"}
